## 作業目標: 了解斷詞演算法的背後計算

### 根據課程講述的內容, 請計算出下列剩餘所有情況的
若有一個人連續觀察到三天水草都是乾燥的(Dry), 則這三天的天氣機率為何？(可參考講義第13頁)
(Hint: 共有6種可能機率)

```python
states = ('sunny', 'rainy')
observations = ('dry', 'damp', 'soggy')
start_probability = {'sunny': 0.4, 'rainy': 0.6}
transition_probability = {'sunny':{'sunny':0.6, 'rainy':0.4},
                          'rainy': {'sunny':0.3, 'rainy':0.7}}
emission_probatility = {'sunny': {'dry':0.6, 'damp':0.3, 'soggy':0.1},
                        'rainy': {'dry':0.1, 'damp':0.4, 'soggy':0.5}}
```

```
觀察狀態 = 'dry', 'dry', 'dry'
Sunny, Sunny, Sunny: 0.4*(0.6)*0.6*(0.6)*0.6*(0.6) = 0.031104
Rainy, Sunny, Sunny: 0.6*(0.1)*0.3*(0.6)*0.6*(0.6) = 0.003888
###<your answers>###
Sunny, Sunny, Sunny: 0.4*(0.6) * 0.6*(0.6) * 0.6*(0.6) = 0.031104
Sunny, Sunny, Rainy: 0.4*(0.6) * 0.6*(0.6) * 0.4*(0.1) = 0.003456
Sunny, Rainy, Sunny: 0.4*(0.6) * 0.4*(0.1) * 0.3*(0.1) = 0.000288
Sunny, Rainy, Rainy: 0.4*(0.6) * 0.4*(0.1) * 0.7*(0.1) = 0.000672
Rainy, Sunny, Sunny: 0.6*(0.1) * 0.3*(0.6) * 0.6*(0.6) = 0.003888
Rainy, Rainy, Sunny: 0.6*(0.1) * 0.7*(0.1) * 0.3*(0.6) = 0.000756
Rainy, Sunny, Rainy: 0.6*(0.1) * 0.3*(0.6) * 0.4*(0.1) = 0.000432
Rainy, Rainy, Rainy: 0.6*(0.1) * 0.7*(0.1) * 0.7*(0.1) = 0.000294

最大機率為: Sunny, Sunny, Sunny
```

### 根據上述條件, 寫出Viterbi應用程式

In [1]:
observations = ('dry', 'dry', 'dry') #實際上觀察到的狀態為dry, dry, dry
states = ('sunny', 'rainy')
start_probability = {'sunny': 0.4, 'rainy': 0.6}
transition_probability = {'sunny':{'sunny':0.6, 'rainy':0.4},
                          'rainy': {'sunny':0.3, 'rainy':0.7}}
emission_probatility = {'sunny': {'dry':0.6, 'damp':0.3, 'soggy':0.1},
                        'rainy': {'dry':0.1, 'damp':0.4, 'soggy':0.5}}

In [34]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    path = {}

    # Initialize base cases (t == 0)
    for y in states:
        ###<your codes>###
        #print(y)
        s1_value = start_p[y] * emit_p[y][obs[0]]
        print('{}, s1_value={}*{}'.format(y,str(start_p[y]),str(emit_p[y][obs[0]])))
        V[0][y] = s1_value
        path[y] = [y]
    print(V, path)
    print('===')
        
    # Run Viterbi for t > 0
    summary_list=[]
    i=1
    for t in range(1,len(obs)-1):
        ###<your codes>###
        V.append({})
        newpath = {}
        for current_state in states:
            for previous_state in states:
                s2_value = trans_p[previous_state][current_state] * emit_p[current_state][obs[t]]
                for next_state in states:
                    s3_value = trans_p[current_state][next_state] * emit_p[next_state][obs[t]]
                    print('{}, {}, {}, {}: {}* {}*{} * {}*{}'.format(str(i),previous_state,current_state,next_state,str(V[0][previous_state]),str(trans_p[previous_state][current_state]),str(emit_p[current_state][obs[t]]),str(trans_p[current_state][next_state]),str(emit_p[next_state][obs[t]])))
                    prob = V[0][previous_state] * s2_value * s3_value
                    cases_dist={}
                    cases_dist['case'] = i
                    cases_dist['states'] = [previous_state,current_state,next_state]
                    cases_dist['prob'] = prob
                    summary_list.append(cases_dist)
                    i+=1
        print('--')
    #print(summary_list) 
    prob =0
    for cases_dist in summary_list:
        for k,v in cases_dist.items():
            if k == 'prob':
                if v > prob:
                    prob = v
                    state = cases_dist['states']
    
    print('Max: ', state, prob)
    return (prob, state)

In [35]:
result = viterbi(observations,
                 states,
                 start_probability,
                 transition_probability,
                 emission_probatility)

sunny, s1_value=0.4*0.6
rainy, s1_value=0.6*0.1
[{'sunny': 0.24, 'rainy': 0.06}] {'sunny': ['sunny'], 'rainy': ['rainy']}
===
1, sunny, sunny, sunny: 0.24* 0.6*0.6 * 0.6*0.6
2, sunny, sunny, rainy: 0.24* 0.6*0.6 * 0.4*0.1
3, rainy, sunny, sunny: 0.06* 0.3*0.6 * 0.6*0.6
4, rainy, sunny, rainy: 0.06* 0.3*0.6 * 0.4*0.1
5, sunny, rainy, sunny: 0.24* 0.4*0.1 * 0.3*0.6
6, sunny, rainy, rainy: 0.24* 0.4*0.1 * 0.7*0.1
7, rainy, rainy, sunny: 0.06* 0.7*0.1 * 0.3*0.6
8, rainy, rainy, rainy: 0.06* 0.7*0.1 * 0.7*0.1
--
Max:  ['sunny', 'sunny', 'sunny'] 0.031103999999999996


In [36]:
result

(0.031103999999999996, ['sunny', 'sunny', 'sunny'])